## Initializing Python

In [1]:
#!/usr/bin/env python
# -*- coding: UTF-8

In [2]:
# import necessary libraries
from bs4 import BeautifulSoup
import re
import os, csv
import shutil
import urllib
from urllib.request import urlopen
from socket import error as SocketError
import errno
import pickle
import pandas as pd
# import httplib2
import requests, contextlib
from urllib.parse import urljoin, urlparse
from requests.auth import HTTPBasicAuth, HTTPDigestAuth
from os.path import splitext

### Define helper functions

In [3]:
def write_file(str, file_name):
    """This function writes a string str to a given file name"""
    with open(file_name, "a") as text_file:
        text_file.write(str)
def save_json(file, school):
    with open(file, 'wb') as outfile:
#         json.dump(school, outfile)
        pickle.dump(school, outfile)
#want the obj back
def get_obj_back(file):
    with open(file,'rb') as f:
        var = pickle.load(f)
    return var

In [4]:
def check(url):
    """ Helper function, check if url is a valid list <- our backup plan
    This functions helps to check the url that has service unavailable issues
    Since status code fails to check this."""
    
    try:
        urlopen(url)
        
    except urllib.error.URLError:
        print(url + " :URLError")
        return False
    except urllib.error.HTTPError:
        print(url +' :HTTPError')
        return False
    except SocketError:
        print(url + 'SocketError')
        return False
    return True

def check_url(url):
    """This functions uses the status code to determine if the link is valid. This resolves
    the links that redirects and most cases of authentication problems"""
    code = "[no code collected]"
    if url == "":
        return False
    try:
        r = requests.get(url, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
        code = r.status_code
        #backup plan for service unavailable issues
        if code == 503:
            return check(url)
        if code < 400:
            return True   
    except:
        pass
    print("Encountered this invalid link: " + str(url) +" ---Error code: " + str(code))
    return False    


### Load in data

In [5]:
micro_sample_cvs = "../data_URAP_etc/micro-sample_Apr17_rev3.csv" # point to current file where charter URLs, etc are kept

In [6]:
df = pd.read_csv(micro_sample_cvs)
school_urls = df["URL"]
school_names = df["School name"]

### Get all the links of a website

In [7]:
def get_children_links(url_parent, hostname, visited, depth, useless):
    """This function recursively gets the children links of a given link"""
    #we have gone through enough levels or visited this link already 
    if depth == 0 or url_parent in visited or url_parent in useless:
        return set()
    if not check_url(url_parent):
        useless.add(url_parent)
        return set()
    
    #get the html page
    #parse into a BS object
    html_page = requests.get(url_parent, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
    soup = BeautifulSoup(html_page.text, "lxml")

    #we visited url_parent, updated into the set
    visited.add(url_parent)
    
    #now checking its children
    for link in soup.findAll('a'):
        #running recursively in a try-except block to prevent broken links break the code
        try:
            pattern = re.compile("((http|ftp)s?://.*?)")
            current_link = link.get('href')
#             print(current_link)
            if not pattern.match(current_link):
                current_link = urljoin(url_parent, current_link)
            
            #check if the link is within the domain (hostname)
            if hostname in current_link:
#                 print(current_link)
                #combine results from its children's links
                get_children_links(current_link, hostname, visited, depth -1, useless)
        except:
            pass
    return visited
#     print(count)

def getLinks(url, depth):
    text,useless = set(), set()
    hostname = urlparse(url).hostname
    return get_children_links(url, hostname, text, depth, useless)
#delete 

### Testing

In [10]:
# test on 1 url
urls = getLinks("https://www.richland2.org/charterhigh/", 3)
count = 0
print("Running test")
print("Here are all the links from: " + "https://www.richland2.org/charterhigh/" )
for e in urls:
    count += 1
    print(e)
print("There are " + str(count)+ " of them")


Encountered this invalid link: https://www.richland2.org/~ ---Error code: 404
Encountered this invalid link: https://www.richland2.org/~ ---Error code: 404
Encountered this invalid link: https://www.richland2.org/~ ---Error code: 404
Encountered this invalid link: https://www.richland2.org/~ ---Error code: 404
Running test
Here are all the links from: https://www.richland2.org/charterhigh/
https://www.richland2.org/charterhigh/news
https://www.richland2.org/CharterHigh/Special-Pages/Site-Translations
https://www.richland2.org/CharterHigh/Student-Registration
https://www.richland2.org/CharterHigh/News/We-re-Hiring-a-Principal
https://www.richland2.org/CharterHigh/About-Our-School/Calendar
https://www.richland2.org/CharterHigh/School-Information/Lab-Hours
https://www.richland2.org/CharterHigh/News
https://www.richland2.org/CharterHigh/Contact-Us
https://www.richland2.org
https://www.richland2.org/charterhigh/About-Our-School/Calendar
https://www.richland2.org/CharterHigh/news
https://www

/admin/x86_64_apps/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


### Now run this method on 300 sites, save as dictionary and store it as a JSON object

In [ ]:
schools = dict()
cap = len(school_names)
# cap = 2
for i in range(cap):
    link, name = school_urls[i], school_names[i]
    #sanity check make sure all links are valid
    if not check_url(link):
        print("WARNING! " + name + " doesnt have a valid link! " + link)
    print("\nGetting links for " + str(i) + " " + school_names[i] + "...")
    #saving school names as keys and list of links as values
    schools.update({name: list(getLinks(link, 3))})
    # TO DO: PRINT NUMBER LINKS GATHERED (as in a "success" kind of statement)
    

Getting links for 0 RICHLAND TWO CHARTER HIGH...
Encountered this invalid link: https://www.richland2.org/~ ---Error code: 404
Encountered this invalid link: https://www.richland2.org/~ ---Error code: 404
Encountered this invalid link: https://www.richland2.org/~ ---Error code: 404
Encountered this invalid link: https://www.richland2.org/~ ---Error code: 404
Getting links for 1 POLK STATE COLLEGE COLLEGIATE HIGH SCHOOL...
Getting links for 2 RIVER CITY SCHOLARS CHARTER ACADEMY...
Encountered this invalid link: https://www.nhaschools.com/schools/rivercity/tel:6162483390 ---Error code: 400
Encountered this invalid link: https://www.nhaschools.com/schools/rivercity/Transparency.aspx ---Error code: 404
Encountered this invalid link: https://www.nhaschools.com/schools/rivercity/Pages/tel:6162483390 ---Error code: 400
Encountered this invalid link: https://www.nhaschools.com/schools/rivercity/Pages/tel:6162483390 ---Error code: 400
Getting links for 3 DETROIT ENTERPRISE ACADEMY...
Encountere

/admin/x86_64_apps/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
ERROR:requests.packages.urllib3.connection:Certificate did not match expected hostname: www.renaissancecharter.org. Certificate: {'subjectAltName': [('DNS', '*.webfaction.com'), ('DNS', 'webfaction.com')], 'subject': ((('commonName', '*.webfaction.com'),),)}



Encountered this invalid link: http://www.renaissancecharter.org/login ---Error code: [no code collected]

ERROR:requests.packages.urllib3.connection:Certificate did not match expected hostname: www.renaissancecharter.org. Certificate: {'subjectAltName': [('DNS', '*.webfaction.com'), ('DNS', 'webfaction.com')], 'subject': ((('commonName', '*.webfaction.com'),),)}



Encountered this invalid link: http://www.renaissancecharter.org/login ---Error code: [no code collected]
Encountered this invalid link: http://www.renaissancecharter.org/support/opportunities ---Error code: 404

...


Encountered this invalid link: http://girlsprep.org/page.cfm?p=670 ---Error code: [no code collected]

ERROR:requests.packages.urllib3.connection:Certificate did not match expected hostname: www.girlsprep.org. Certificate: {'subjectAltName': [('DNS', 'www.publicprep.org'), ('DNS', 'publicprep.org')], 'subject': ((('commonName', 'www.publicprep.org'),),)}



Encountered this invalid link: http://girlsprep.org/page.cfm?p=669 ---Error code: [no code collected]

ERROR:requests.packages.urllib3.connection:Certificate did not match expected hostname: www.girlsprep.org. Certificate: {'subjectAltName': [('DNS', 'www.publicprep.org'), ('DNS', 'publicprep.org')], 'subject': ((('commonName', 'www.publicprep.org'),),)}



Encountered this invalid link: http://girlsprep.org/page.cfm?p=687 ---Error code: [no code collected]

In [ ]:
save_json("list_of_links", schools)

In [ ]:
school_list = get_obj_back("list_of_links")

In [ ]:
for i in school_list:
    print(i + " HAS THESE " + str(len(school_list[i])) + "EMBEDDED LINKS:" )
    print(school_list[i], "\n")

#### Now we can get all the html text by runing requests to get all the content. 

#### But first, let's define and test functions for getting the PDFs and images, too!

### Get all images

Since we all have a list of the links alreay, now we are ready to create folders to store these images

In [ ]:
def get_images(url):

    html_page = requests.get(url, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
    soup = BeautifulSoup(html_page.text, "lxml")
    images = set()
    # getting the links of the images
    for link in soup.find_all('img'):
        try:
            image = link.get("src")          
            pattern = re.compile("((http|ftp)s?://.*?)")
            #relative link, need to append to parent's link to get the correct link
            if not pattern.match(image):
                image = urljoin(url, image)
            images.add(image)
        except:
            pass

    return images  
def download_image(url, k):
    try:
        response = requests.get(url,stream=True, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
        ext = url[url.rindex("."):]
#         print(ext)
#         file_name = url[url.rindex("/")+1: url.rindex("."+ ext)] +"."+ ext
#         print(file_name)
#         if os.path.exists(file_name):
#             file_name = file_name[:file_name.rindex("."+ ext)] +"_1."+ ext
        with open("img" + str(k)+ "."+ ext, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
            del response
    except:
        print("couldn't get " + url)

### Testing

In [ ]:
r  = get_images("http://www.wattslearningcenter.org/#content_main")
k = 1
os.chdir("../data_URAP_etc/tex2")
for i in r:
    print(i)
    download_image(i, k)
    k += 1
#might need to check on size

### Get pdfs

In [ ]:
def get_pdfs(url):

    html_page = requests.get(url, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
    soup = BeautifulSoup(html_page.text, "lxml")
    pdfs = set()
    # getting the links of the images
    for link in soup.findAll('a'):
        try:
            pdf = link.get('href')          
            pattern = re.compile("((http|ftp)s?://.*?)")
            #relative link, need to append to parent's link to get the correct link
            if not pattern.match(pdf):
                pdf = urljoin(url, pdf)
                
            if(pdf.endswith(".pdf")):
                pdfs.add(pdf)
        except:
            pass

    return pdfs 
def download_pdf(url):
    try:
        
        file_name = url[url.rindex("/")+1: url.rindex(".pdf")] +".pdf"
#         print(file_name)
        if os.path.exists(file_name):
            file_name = file_name[:file_name.rindex(".pdf")] +"_1.pdf"
        response = requests.get(url,stream=True, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
        with open(file_name, 'wb') as f:
            f.write(response.content)
    except:
        print("couldn't get " + url)

### Testing

In [ ]:
m  = get_pdfs("http://inst.eecs.berkeley.edu/~ee16b/sp17/")
os.chdir("../data_URAP_etc/tex2")
j = 0
for i in m:
    print(i)
    download_pdf(i)
    j += 1
print("\nCaptured " + str(j) + " PDF files")

Here is the fun part. We will run our code to get everything, from text, images, and pdfs

In [ ]:
def format_folder_name (k, name):
    """Format a folder nicely for easy access"""
    if k < 10: # Add two zeros to the folder name if k is less than 10 (for ease of organizing the output folders)
        dirname = "00" + str(k) + " " + name
    elif k < 100: # Add one zero if k is less than 100
        dirname = "0" + str(k) + " " + name
    else: # Add nothing if k>100
        dirname = str(k) + " " + name
    return dirname

In [ ]:
school_list

In [ ]:
def run_master(school_list, parent_folder):
    """webscrape links and print output to appropriate folders"""
    #navigate to parent folder
    k = 0
    os.chdir(parent_folder)
    for k, school in enumerate(school_list):
        # create dir my_folder if it doesn't exist yet
        os.chdir(parent_folder)
        curr_folder_name = format_folder_name(k, school)
        if not os.path.exists(curr_folder_name):
            os.makedirs(curr_folder_name)
        #navigate to the correct folder, ready to scrap
        os.chdir(curr_folder_name)
        
        #scrap link to get html content
        for url in school_list[school]:
            html_page = requests.get(url, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
            soup = BeautifulSoup(html_page.text, "lxml")
            try:
                #beautify text name for easy access, accept alphanumeric letters only
                #name is the title of the page
                file_name = re.sub('[^A-Za-z0-9]+', '', soup.title.string)
            except:
                file_name = "no name"
#                 i += 1
            #create folder if not exists & go to that folder
            if not os.path.exists(file_name):
                os.makedirs(file_name)
            os.chdir(file_name)
            
            #format text as follow 
            #first line: url, second line :-------------(separater), third line - end: scraping result
            write_file("URL:" + str(url)+ "\n","text.txt" )
            write_file("----------------------------------------------\n", "text.txt")
            write_file(str(soup.prettify()), "text.txt")
            
            #now get all the images
            list_images = get_images(url)
            for num,img in enumerate(list_images):
                download_image(img, num)
            #now get all the pdfs
            list_pdf = get_pdfs(url)  
            for pdf in list_pdf:
                download_pdf(pdf)
                
            #done, go back to parent directory, ready to scrape the next url
            os.chdir('..')
        
        k += 1

In [ ]:
parent_folder = "/nobackup/jhaber/requests micro-sample"
run_master(school_list, parent_folder)

In [ ]:
school_list['DETROIT COMMUNITY SCHOOLS-HIGH SCHOOL']